# **🔍 ¿Existe Relación Cruzada? La Causalidad de Granger**

La prueba estadística más utilizada para determinar si una serie temporal ayuda a predecir otra se llama Test de Causalidad de Granger (Granger Causality Test).

### **¿Qué hace la prueba?**

Esta prueba evalúa si los valores rezagados de la serie $Y_t$ (Temp_Minima) mejoran la predicción de la serie $X_t$ (Temp_Maxima), más allá de lo que $X_t$ puede predecir por sí misma :

1. Hipótesis Nula ($H_0$) ➡ La serie $Y_t$ NO causa Granger a la serie $X_t$. (No hay relación predictiva).

2. Hipótesis Alternativa ($H_A$) ➡ La serie $Y_t$ SÍ causa Granger a la serie $X_t$. (Hay relación predictiva).

**Regla de Decisión:** Si el p-valor de la prueba es menor que $\alpha=0.05$, se rechaza $H_0$ y se concluye que la relación predictiva cruzada es estadísticamente significativa.

### **💻 Código para el Test de Causalidad de Granger**
El Test de Causalidad de Granger se debe aplicar a las series estacionarias que creaste (las Box-Cox Diferenciadas), ya que la prueba asume que las series son estacionarias.

In [2]:
import pandas as pd
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.stattools import grangercausalitytests

# --- 1. Cargar las Series Estacionarias ---
try:
    # Cargamos el archivo estacionario que acabas de guardar
    df_stationary = pd.read_csv('dataset/datos_temperaturas_estacionarios.csv', index_col='Fecha', parse_dates=True)
except FileNotFoundError:
    print("Error: Asegúrate de que 'datos_temperaturas_estacionarios.csv' exista.")
    # Si no existe, puedes recrear df_stationary a partir del código de Transformaciones.ipynb

In [3]:
# --- 2. Preparar los datos para el Test de Granger ---

# Seleccionamos las columnas de interés y las aseguramos como un array bidimensional.
data_granger = df_stationary[['Diff_BoxCox_Max', 'Diff_BoxCox_Min']]

# Se necesita especificar el número máximo de rezagos (lags) a probar.
# Un lag máximo de 7 (una semana) o 30 (un mes) es común para detectar interdependencias de corto plazo.
max_lags = 7 

print(f"===============================================================")
print(f"   TEST DE CAUSALIDAD DE GRANGER (Máx. Lags = {max_lags})")
print(f"===============================================================")

   TEST DE CAUSALIDAD DE GRANGER (Máx. Lags = 7)


In [5]:
# --- 3. Test 1: ¿Minima causa a Maxima? (Y -> X) ---
print("\n[A] H0: Temp Mínima NO causa Granger a Temp Máxima")
granger_result_min_to_max = grangercausalitytests(
    data_granger[['Diff_BoxCox_Max', 'Diff_BoxCox_Min']], # Orden: [Variable_a_predecir, Variable_causante]
    maxlag=max_lags, 
    verbose=False
)

# Analizar los resultados del test de F-estadístico para el lag óptimo (generalmente el último)
# Seleccionamos el p-valor de F-test del último lag probado (max_lags)
p_value_min_to_max = granger_result_min_to_max[max_lags][0]['ssr_ftest'][1]
print(f"   P-valor (Min -> Max): {p_value_min_to_max:.5f}")

if p_value_min_to_max < 0.05:
    print("   ➡️ Conclusión: Rechazamos H0. ¡La Mínima SÍ ayuda a predecir la Máxima!")
else:
    print("   ➡️ Conclusión: No se rechaza H0. No hay evidencia de que la Mínima prediga la Máxima.")


# --- 4. Test 2: ¿Maxima causa a Minima? (X -> Y) ---
print("\n[B] H0: Temp Máxima NO causa Granger a Temp Mínima")
granger_result_max_to_min = grangercausalitytests(
    data_granger[['Diff_BoxCox_Min', 'Diff_BoxCox_Max']], # Orden: [Variable_a_predecir, Variable_causante]
    maxlag=max_lags, 
    verbose=False
)

p_value_max_to_min = granger_result_max_to_min[max_lags][0]['ssr_ftest'][1]
print(f"   P-valor (Max -> Min): {p_value_max_to_min:.5f}")

if p_value_max_to_min < 0.05:
    print("   ➡️ Conclusión: Rechazamos H0. ¡La Máxima SÍ ayuda a predecir la Mínima!")
else:
    print("   ➡️ Conclusión: No se rechaza H0. No hay evidencia de que la Máxima prediga la Mínima.")

print("\n===============================================================")


[A] H0: Temp Mínima NO causa Granger a Temp Máxima
   P-valor (Min -> Max): 0.39372
   ➡️ Conclusión: No se rechaza H0. No hay evidencia de que la Mínima prediga la Máxima.

[B] H0: Temp Máxima NO causa Granger a Temp Mínima
   P-valor (Max -> Min): 0.00000
   ➡️ Conclusión: Rechazamos H0. ¡La Máxima SÍ ayuda a predecir la Mínima!



c:\Users\delhy.py\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\delhy.py\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1545: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


### **📝 Conclusiones Clave del Test de Causalidad de Granger**

El Test de Causalidad de Granger se aplicó correctamente a las series estacionarias (Box-Cox Diferenciadas) con un máximo de 7 rezagos (lags).

* Test [A]: ¿Temp Mínima (ayer) causa a Temp Máxima (hoy)?

1. Resultado ➡ **No Causalidad**
2. P-valor (Min $\rightarrow$ Max) ➡ $0.39372$
3. Decisión ➡ No se rechaza $H_0$
4. Conclusión ➡ No hay evidencia estadística de que la temperatura Mínima de días anteriores ayude a predecir la Máxima de hoy.

* Test [B]: ¿Temp Máxima (ayer) causa a Temp Mínima (hoy)?

1. Resultado ➡ Causalidad Fuerte
2. P-valor (Min $\rightarrow$ Max) ➡ $0.00000$
3. Decisión ➡ Se rechaza $H_0$
4. Conclusión ➡ La Máxima SÍ ayuda a predecir la Mínima. El p-valor es muy significativo ($p < 0.05$).

### 🎯 Justificación Final para el Modelado Vectorial
La conclusión es clara: **Existe una relación causal unilateral (unidireccional) estadísticamente significativa** en el sistema de temperaturas.

Implicación para el Modelo:

1. **Decisión Confirmada**: Dado que la serie Temp_Maxima es un predictor significativo para la serie Temp_Minima, **es absolutamente necesario** utilizar un modelo vectorial.

2. **Modelo a Usar**: El modelo **SARIMAX Vectorial (VARIMA)** es la elección correcta, ya que está diseñado para capturar y explotar esta interdependencia cruzada al realizar predicciones.

Si se hubieran usado dos modelos SARIMA univariantes independientes, el modelo de la Temperatura Mínima habría ignorado la valiosa información predictiva contenida en los rezagos de la Temperatura Máxima, resultando en predicciones menos precisas.

Si se hubieran usado dos modelos SARIMA univariantes independientes, el modelo de la Temperatura Mínima habría ignorado la valiosa información predictiva contenida en los rezagos de la Temperatura Máxima, resultando en predicciones menos precisas.